<a href="https://colab.research.google.com/github/Delonix7/Data_Science_Essentials/blob/main/Web_Scraping_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install bs4 #Installs the package needed for web scraping
# !pip install beautifulsoup4
# to check the version of BeautifulSoup, import bs4 and run bs4.__version__

In [2]:
from bs4 import BeautifulSoup as bs

Few things to note in the SIMPLE_HTML string below, which essentially applies to all HTML documents. h1 = Header, p = paragraph, ul = unordered list, li = list item

In [11]:
SIMPLE_HTML ='''<html>
<head></head>
<body>
<h1>This is a title</h1>
<p class="subtitle">Lorem ipsum dolor sit amet. Consectetur edipiscim elit.</p>
<p>Here's another p without a class</p>
<ul>
  <li>Rolf</li>
  <li>Charlie</li>
  <li>Jen</li>
  <li>Jose</li>
</ul>
</body>
</html>'''

In [23]:
# with open ('simple_html.html') as doc: #if html is save in a file, use this code to parse it to BeautifulSoup
#   soup = bs(doc, 'html.parser')

In [25]:
simple_soup = bs(SIMPLE_HTML, 'html.parser') # first argument is the html file or  string to be parsed.
#second argument is the type of document being parsed.

In [33]:
print(simple_soup.find('h1')) #goes throught the html document and find the first h1 tag
#use find_all() if you want to find all h1 tags

<h1>This is a title</h1>


In [36]:
print(simple_soup.find('h1').string) #adding .string prints out the content of the tag

This is a title


In [ ]:
for f in simple_soup.find_all('li'): # to access the contents of tags found with find_all(), loop over the contents
  print(f.string)

## Let's write some functions to search the html document

In [59]:
def find_title(): #searches for the first h1 tag and returns its content as a string
  h1_tag = simple_soup.find('h1')
  return h1_tag.string

In [72]:
find_title()

'This is a title'

In [60]:
def find_list_items(): #searches for all the li tags and returns it as a list of strings
  list_items = simple_soup.find_all('li')
  list_contents = [items.string for items in list_items]
  return list_contents

In [73]:
find_list_items()

['Rolf', 'Charlie', 'Jen', 'Jose']

In [63]:
def find_subtitle(): #searching for a specific tag using a class attribute
  paragraph = simple_soup.find('p', {'class': 'subtitle'}) #pass the class attribute to find as a dictionary
  return paragraph.string

In [74]:
find_subtitle()

'Lorem ipsum dolor sit amet. Consectetur edipiscim elit.'

In [70]:
def find_other_paragraph(): #searches for all paragraphs without the attribute class = 'subtitle'
  paragraphs = simple_soup.find_all('p') #finds all p tags
  other_paragraph = [p.string for p in paragraphs if 'subtitle' not in p.attrs.get('class', [])]
  # creates a list with list comprehension. Loops over the contents of paragraphs and finds the paragraphs
  # with subtitle not in p.attrs.get('class', []) --> the get method returns the value for the key 'class'.
  #if the key 'class' is not found, it returns None. The second arg [] will be returned if value is None.
  return other_paragraph

In [75]:
find_other_paragraph()

["Here's another p without a class"]